In [1]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=bcf593718859fc0a2f06b8dddf53664023e6e788cb398c4cc6bcad0a41e00444
  Stored in directory: /root/.cache/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [3]:
!cd /usr/local/lib/python3.12/dist-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.30/redshift-jdbc42-2.1.0.30.jar

--2025-11-21 03:49:28--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.30/redshift-jdbc42-2.1.0.30.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.174.232, 16.15.193.176, 52.217.71.86, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.174.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1088404 (1.0M) [application/java-archive]
Saving to: ‘redshift-jdbc42-2.1.0.30.jar’

redshift-jdbc42-2.1 100%[===================>]   1.04M  --.-KB/s    in 0.01s   

2025-11-21 03:49:28 (88.1 MB/s) - ‘redshift-jdbc42-2.1.0.30.jar’ saved [1088404/1088404]



In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark DataFrame #5") \
    .getOrCreate()

## Redshift와 연결해서 테이블들을 데이터프레임으로 로딩하기

In [ ]:
df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=your-username&password=your-password") \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

In [ ]:
df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=your-username&password=your-password") \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

In [7]:
df_user_session_channel.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)



In [8]:
df_user_session_channel.rdd.getNumPartitions()

1

In [9]:
df_session_timestamp.printSchema()

root
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [10]:
df_session_timestamp.rdd.getNumPartitions()

1

## DataFrame으로 처리하기

In [11]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")

In [12]:
session_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [13]:
session_df.show(5)

+------+--------------------+--------+--------------------+--------------------+
|userid|           sessionid| channel|           sessionid|                  ts|
+------+--------------------+--------+--------------------+--------------------+
|  1501|0135456d6a3c1051f...|  Google|0135456d6a3c1051f...|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...|Facebook|01a416a7e28d0d229...|2019-05-26 14:23:...|
|  2133|02ea66ee24d57e285...| Organic|02ea66ee24d57e285...| 2019-10-28 22:50:40|
|  1961|0302915889fa38fe5...| Youtube|0302915889fa38fe5...| 2019-11-29 15:16:49|
|   599|03ce0331cbd983f16...| Organic|03ce0331cbd983f16...|2019-07-18 13:49:...|
+------+--------------------+--------+--------------------+--------------------+
only showing top 5 rows



In [14]:
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner").select(
    "userid", "sessionid", "channel", "ts"
)

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `sessionid` is ambiguous, could be: [`sessionid`, `sessionid`].

In [15]:
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner").select(
    "userid", df_user_session_channel.sessionid, "channel", "ts"
)

In [16]:
session_df.show(5)

+------+--------------------+--------+--------------------+
|userid|           sessionid| channel|                  ts|
+------+--------------------+--------+--------------------+
|  1501|0135456d6a3c1051f...|  Google|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...|Facebook|2019-05-26 14:23:...|
|   243|0226aa5193c66d990...|  Google|2019-07-01 23:04:...|
|  2776|029bf49b584c641f0...|Facebook|2019-11-11 20:37:...|
|   939|02b8d6c2775b756de...|  Google|2019-09-01 15:29:...|
+------+--------------------+--------+--------------------+
only showing top 5 rows



In [17]:
channel_count_df = session_df.groupby("channel").count().orderBy("count", ascending=False)

In [18]:
channel_count_df.show()

+---------+-----+
|  channel|count|
+---------+-----+
|  Youtube|17091|
|   Google|16982|
|    Naver|16921|
|  Organic|16904|
|Instagram|16831|
| Facebook|16791|
+---------+-----+



In [19]:
from pyspark.sql.functions import date_format, asc, countDistinct

session_df.withColumn('month', date_format('ts', 'yyyy-MM')).groupby('month').\
    agg(countDistinct("userid").alias("mau")).sort(asc('month')).show()

+-------+---+
|  month|mau|
+-------+---+
|2019-05|281|
|2019-06|459|
|2019-07|623|
|2019-08|662|
|2019-09|639|
|2019-10|763|
|2019-11|721|
+-------+---+



## Spark SQL로  처리하기

In [20]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")

In [21]:
df_session_timestamp.createOrReplaceTempView("session_timestamp")

In [22]:
channel_count_df = spark.sql("""
    SELECT channel, count(distinct userId) uniqueUsers
    FROM session_timestamp st
    JOIN user_session_channel usc ON st.sessionID = usc.sessionID
    GROUP BY 1
    ORDER BY 1
""")

In [23]:
channel_count_df

DataFrame[channel: string, uniqueUsers: bigint]

In [24]:
channel_count_df.show()

+---------+-----------+
|  channel|uniqueUsers|
+---------+-----------+
| Facebook|        889|
|   Google|        893|
|Instagram|        895|
|    Naver|        882|
|  Organic|        895|
|  Youtube|        889|
+---------+-----------+



In [25]:
mau_df = spark.sql("""
SELECT
  LEFT(A.ts, 7) AS month,
  COUNT(DISTINCT B.userid) AS mau
FROM session_timestamp A
JOIN user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1
ORDER BY 1 DESC""")

In [26]:
mau_df.collect()

[Row(month='2019-11', mau=721),
 Row(month='2019-10', mau=763),
 Row(month='2019-09', mau=639),
 Row(month='2019-08', mau=662),
 Row(month='2019-07', mau=623),
 Row(month='2019-06', mau=459),
 Row(month='2019-05', mau=281)]